In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

from keras.preprocessing.image import array_to_img, img_to_array, load_img#,save_img

from sklearn.model_selection import train_test_split

from dataset import pad_reflect, unpad_reflect

#from torchvision import transforms

SEED = 6789#5678#4567#3456#2345#1234

img_size_ori = 101
img_size_target = 101#128

/home/endi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Loading of training/testing ids and depths
train_df = pd.read_csv("../data/raw/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../data/raw/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

print(len(train_df), len(test_df))

# Loading images into numpy array
train_df["images"] = [np.array(load_img("../data/raw/train_images/images/{}.png".format(idx), color_mode = "grayscale")) / 255 for idx in tqdm_notebook(train_df.index)]
train_df["masks"] = [np.array(load_img("../data/raw/train_images/masks/{}.png".format(idx), color_mode = "grayscale")) / 255 for idx in tqdm_notebook(train_df.index)]

4000 18000


In [3]:
train_df["coverage_pct"] = train_df.masks.map(np.sum) / img_size_ori**2
train_df["coverage"] = train_df.masks.map(np.sum)

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df["coverage_pct"].map(cov_to_class)

In [4]:
# delete images that has < 1 pixel mask in total
#train_df.head(2)
#train_df = train_df.loc[train_df['coverage']>0, ]

#print('zero mask pct: ', (train_df.coverage==0).mean())#0.39

In [4]:
# Create train/validation split stratified by salt coverage
# ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
#     train_df.index.values,
#     np.array(train_df.images.map(pad_reflect).tolist()).reshape(-1, 1, img_size_target, img_size_target), 
#     np.array(train_df.masks.map(pad_reflect).tolist()).reshape(-1, 1, img_size_target, img_size_target), 
#     train_df.coverage.values,
#     train_df.z.values,
#     test_size=0.1, stratify=train_df.coverage_class, random_state=SEED)

ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
    train_df.index.values,
    np.array(train_df.images.tolist()).reshape(-1, 1, img_size_target, img_size_target), 
    np.array(train_df.masks.tolist()).reshape(-1, 1, img_size_target, img_size_target), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.1, stratify=train_df.coverage_class, random_state=SEED)

#Data augmentation
#x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
#y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)
print(x_train.shape)
print(y_valid.shape)

(3600, 1, 101, 101)
(400, 1, 101, 101)


In [5]:
import pickle
with open('../data/processed/dataset_%d.pkl'%SEED, 'wb') as f:#dataset_aug_%d.pkl
    pickle.dump([ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test], f)

# with open('../data/processed/dataset_aug_%d.pkl'%SEED, 'rb') as f:
#     ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = pickle.load(f)
print(x_train.shape)
print(y_valid.shape)

(3600, 1, 101, 101)
(400, 1, 101, 101)


In [7]:
cnt = 0
for i in range(y_train.shape[0]):
    if y_train[i,0,].sum()<25:
        print(i)
        cnt += 1

0
3
6
10
13
17
19
20
23
26
28
31
32
36
38
39
40
42
43
44
45
46
48
50
53
54
56
58
63
64
65
73
74
79
82
83
84
90
96
98
102
103
105
106
110
113
114
116
118
121
122
123
126
128
129
130
133
138
141
142
143
144
145
149
150
154
156
158
161
162
163
164
165
166
172
174
175
187
190
191
192
194
201
202
204
205
212
216
217
230
231
235
236
237
238
241
244
247
249
250
251
252
255
257
259
261
262
265
268
270
271
273
275
276
286
288
289
290
291
294
296
298
301
302
303
305
310
314
322
327
328
329
333
335
336
343
346
348
354
355
360
361
371
374
377
380
388
389
390
391
393
398
400
405
410
411
414
421
422
425
430
433
435
437
439
440
441
443
444
445
448
449
451
452
453
456
458
462
463
464
467
472
473
477
479
483
484
486
489
492
494
496
497
498
503
504
509
517
519
521
522
523
527
529
530
534
535
536
539
541
542
547
551
555
558
559
560
561
562
563
564
570
571
574
575
578
580
581
582
584
588
590
591
592
597
600
601
604
607
608
616
622
628
629
630
633
639
642
645
647
651
653
654
657
658
661
663
667
668
669
670

In [9]:
cnt / 3600

0.39861111111111114